# Top 5 countries with the sharpest decline in birth rates since 2019


## Importing the data

In [1]:
import pandas as pd

# Import data to dataframe
df = pd.read_csv('data/fertility_global_wide-raw.csv')

## Analysis of dataset shape

In [2]:
# Datasets sourced from World Bank often have many metadata columns
print(df.shape)
print(df.columns[:10])
df.head()

(265, 105)
Index(['STRUCTURE', 'STRUCTURE_ID', 'ACTION', 'FREQ', 'REF_AREA', 'INDICATOR',
       'SEX', 'AGE', 'URBANISATION', 'UNIT_MEASURE'],
      dtype='str')


,STRUCTURE,STRUCTURE_ID,ACTION,FREQ,REF_AREA,INDICATOR,SEX,AGE,URBANISATION,UNIT_MEASURE,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,datastructure,WB.DATA360:DS_DATA360(1.3),I,A,TCA,WB_WDI_SP_DYN_TFRT_IN,_T,_T,_T,BR_W,...,1.7290,1.7180,1.7130,1.7040,1.6770,1.6170,1.5520,1.501,1.4770,1.4630
1,datastructure,WB.DATA360:DS_DATA360(1.3),I,A,FRO,WB_WDI_SP_DYN_TFRT_IN,_T,_T,_T,BR_W,...,2.5767,2.4278,2.6393,2.4926,2.5118,2.4341,2.3498,2.325,2.0713,1.8586
2,datastructure,WB.DATA360:DS_DATA360(1.3),I,A,ALB,WB_WDI_SP_DYN_TFRT_IN,_T,_T,_T,BR_W,...,1.7210,1.6310,1.5550,1.4860,1.4150,1.3950,1.3710,1.365,1.3550,1.3480
3,datastructure,WB.DATA360:DS_DATA360(1.3),I,A,SGP,WB_WDI_SP_DYN_TFRT_IN,_T,_T,_T,BR_W,...,1.2500,1.2400,1.2000,1.1600,1.1400,1.1400,1.1000,1.120,1.0400,0.9700
4,datastructure,WB.DATA360:DS_DATA360(1.3),I,A,BOL,WB_WDI_SP_DYN_TFRT_IN,_T,_T,_T,BR_W,...,2.9470,2.8860,2.8290,2.7770,2.7300,2.6880,2.6510,2.618,2.5840,2.5470


In [3]:
df.info(verbose=True)

<class 'pandas.DataFrame'>
RangeIndex: 265 entries, 0 to 264
Data columns (total 105 columns):
 #    Column                  Dtype  
---   ------                  -----  
 0    STRUCTURE               str    
 1    STRUCTURE_ID            str    
 2    ACTION                  str    
 3    FREQ                    str    
 4    REF_AREA                str    
 5    INDICATOR               str    
 6    SEX                     str    
 7    AGE                     str    
 8    URBANISATION            str    
 9    UNIT_MEASURE            str    
 10   COMP_BREAKDOWN_1        str    
 11   COMP_BREAKDOWN_2        str    
 12   COMP_BREAKDOWN_3        str    
 13   AGG_METHOD              str    
 14   UNIT_TYPE               str    
 15   DECIMALS                int64  
 16   DATABASE_ID             str    
 17   TIME_FORMAT             str    
 18   COMMENT_TS              str    
 19   UNIT_MULT               int64  
 20   DATA_SOURCE             str    
 21   OBS_CONF                s

## Converting the dataset to long format

In [24]:
# To ensure usable data it is neccessary to convert the existing dataset to a long format
df_long = df.melt(
    id_vars=["REF_AREA_LABEL"],
    # filter the data for data from 2000 to 2023
    value_vars=[str(year) for year in range(2000, 2024)],
    var_name="Year",
    value_name="Fertility"
)
df_long['Year'] = df_long['Year'].astype(int)
df_long.head()

,REF_AREA_LABEL,Year,Fertility
0,Turks and Caicos Islands,2000,2.4060
1,Faroe Islands,2000,2.5861
2,Albania,2000,2.2170
3,Singapore,2000,1.6000
4,Bolivia,2000,3.9910


In [5]:
df_long.describe()

,Year,Fertility
count,6360.000000,6360.000000
mean,2011.500000,2.834922
std,6.922731,1.456570
min,2000.000000,0.586000
25%,2005.750000,1.709880
50%,2011.500000,2.324037
75%,2017.250000,3.711250
max,2023.000000,7.829000


In [6]:
pd.isna(df_long['Year']).sum()

np.int64(0)

In [7]:
pd.isna(df_long['Fertility']).sum()

np.int64(0)

In [17]:
df_long = df_long.sort_values(by=['REF_AREA_LABEL', 'Year', 'Fertility'])
df_long.head()

,REF_AREA_LABEL,Year,Fertility
156,Afghanistan,2000,7.566
421,Afghanistan,2001,7.453
686,Afghanistan,2002,7.320
951,Afghanistan,2003,7.174
1216,Afghanistan,2004,7.018


In [25]:
start = df_long[df_long["Year"] == 2019][["REF_AREA_LABEL", "Fertility"]].copy()
end   = df_long[df_long["Year"] == 2023][["REF_AREA_LABEL", "Fertility"]].copy()

merged = start.merge(
    end,
    on="REF_AREA_LABEL",
    suffixes=("_2019", "_2023")
)

print(start.shape)
print(end.shape)

merged.head()

(265, 2)
(265, 2)


,REF_AREA_LABEL,Fertility_2019,Fertility_2023
0,Turks and Caicos Islands,1.6170,1.4630
1,Faroe Islands,2.4341,1.8586
2,Albania,1.3950,1.3480
3,Singapore,1.1400,0.9700
4,Bolivia,2.6880,2.5470


In [27]:
merged["Percent_Change"] = (
    (merged["Fertility_2023"] - merged["Fertility_2019"])
    / merged["Fertility_2019"]
) * 100

merged.head()

,REF_AREA_LABEL,Fertility_2019,Fertility_2023,Percent_Change
0,Turks and Caicos Islands,1.6170,1.4630,-9.523810
1,Faroe Islands,2.4341,1.8586,-23.643236
2,Albania,1.3950,1.3480,-3.369176
3,Singapore,1.1400,0.9700,-14.912281
4,Bolivia,2.6880,2.5470,-5.245536


In [28]:
decline = merged.sort_values("Percent_Change")
top5_decline = decline.head(5)

top5_decline.head()

,REF_AREA_LABEL,Fertility_2019,Fertility_2023,Percent_Change
75,"Macao SAR, China",0.899,0.586,-34.816463
218,China,1.496,0.999,-33.221925
159,"Hong Kong SAR, China",1.064,0.751,-29.417293
153,Kuwait,2.082,1.524,-26.801153
47,Curaçao,1.600,1.200,-25.000000
